In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
df = pd.read_csv('data/train/train_prepared.csv', header = 0)

In [4]:
df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,"('purchase_amount', 'min')","('purchase_amount', 'max')","('purchase_amount', 'mean')","('purchase_amount', 'median')","('purchase_amount', 'std')","('purchase_amount', 'size')","('purchase_amount', 'sum')","('month_lag', 'min')","('month_lag', 'mean')","('month_lag', 'median')","('month_lag', 'std')"
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,-0.739395,2.258395,-0.638341,-0.698042,0.212139,260,-165.968739,-8,-3.911538,-4.0,2.397687
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,-0.742400,4.630299,-0.600018,-0.708590,0.384967,350,-210.006336,-12,-5.031429,-5.0,3.804934
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,-0.730138,-0.145847,-0.678311,-0.698868,0.087380,43,-29.167391,-13,-8.604651,-10.0,3.842987
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,-0.740897,1.445596,-0.642745,-0.707989,0.261624,77,-49.491364,-5,-2.831169,-4.0,1.802065
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,-0.746156,7.193041,-0.366073,-0.689807,1.352094,133,-48.687656,-3,-1.285714,-1.0,1.026700


**Группировка**

Расширим категориальные признаки на бинарные по их уникальным сочетаниям (все перестановки)

In [6]:
df_gr = df

for f1 in [1,2,3,4,5]:
    for f2 in [1,2,3]:
        for f3 in [0,1]:
            col_name = 'feature_' + str(f1) + str(f2) + str(f3)
            df_gr[col_name] = [0] * 201917

In [116]:
for idx, _ in enumerate(df_gr.index):
    cur_val = df_gr.loc[idx]
    col_name = 'feature_' + str(cur_val[2]) + str(cur_val[3]) + str(cur_val[4])
    df_gr.loc[idx, col_name] = 1

In [112]:
df_gr.shape

(201917, 36)

In [118]:
df_gr.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,feature_110,feature_111,feature_120,feature_121,...,feature_420,feature_421,feature_430,feature_431,feature_510,feature_511,feature_520,feature_521,feature_530,feature_531
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
df_gr.loc[120]

first_active_month            2017-08
card_id               C_ID_0510accad2
feature_1                           3
feature_2                           1
feature_3                           1
target                       0.454811
feature_110                         0
feature_111                         0
feature_120                         0
feature_121                         0
feature_130                         0
feature_131                         0
feature_210                         0
feature_211                         0
feature_220                         0
feature_221                         0
feature_230                         0
feature_231                         0
feature_310                         0
feature_311                         1
feature_320                         0
feature_321                         0
feature_330                         0
feature_331                         0
feature_410                         0
feature_411                         0
feature_420 

Разложим дату на признаки месяца и года

In [121]:
df_gr.first_active_month = pd.to_datetime(df_gr.first_active_month)
df_gr['yr'] = df_gr.first_active_month.dt.year
df_gr['mnth'] = df_gr.first_active_month.dt.month

***

Подготовим векторные представления

In [122]:
X1 = np.array(df.iloc[:, 2:5 ])
X2 = np.array(df.iloc[:, 6: ])
X = np.column_stack((X1, X2))
y = np.array(df.loc[:, ['target']])

In [123]:
X.shape

(201917, 35)

In [124]:
y.shape

(201917, 1)

# Модель 

Разобьем модель на обучающую и тестовую выборку

In [125]:
from sklearn.model_selection import train_test_split

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

Обучим модель

In [127]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Рассчитаем среднеквадратичную ошибку и коэффициент детерминации 

In [128]:
import math
math.sqrt(mean_squared_error(y_test, y_pred)) 

3.7952205138803037

In [129]:
r2_score(y_test, y_pred)

0.003221936569715478

Уравнение совершенно не описывает дисперсии остатков, наверное это связано с низкой степенью влияния на целевую функцию выбранного факторного пространства.